# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [31]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from keys import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [8]:
symbol='AAPL'
api_url = f"https://cloud.iexapis.com/stable/stock/{symbol}/quote?token=pk_32ef64b2003542b6a829b8f94831c789"
data = requests.get(api_url).json()
print(data)


{'avgTotalVolume': 57952437, 'calculationPrice': 'close', 'change': -0.37, 'changePercent': -0.00206, 'close': None, 'closeSource': 'official', 'closeTime': None, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': None, 'delayedPriceTime': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPrice': None, 'extendedPriceTime': None, 'high': None, 'highSource': '15 minute delayed price', 'highTime': 1686081599992, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 178.96, 'iexBidSize': 100, 'iexClose': 179.19, 'iexCloseTime': 1686081599602, 'iexLastUpdated': 1686081599602, 'iexMarketPercent': 0.01771495458004853, 'iexOpen': 180.02, 'iexOpenTime': 1686058201333, 'iexRealtimePrice': 179.19, 'iexRealtimeSize': 200, 'iexVolume': 1148786, 'lastTradeTime': 1686081599967, 'latestPrice': 179.21, 'latestSource': 'Close', 'latestTime': 'June 6, 2023', 'latestUpdate': 1686081600348, 'latestVolume': None, 'low': None, 'lowSource': '15 minute delayed price', 'lowTime': 168

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data['latestPrice']
market_cap = data["marketCap"]
market_cap

2818740685420

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ["Ticker", "Stock Price", "Markert Capitalization", "Number of Share to Buy"]

In [7]:
final_df = pd.DataFrame(columns = my_columns)
final_df.append(
        pd.Series([
            symbol, price, market_cap, "N/A"
        ],index = my_columns), ignore_index=True
)

/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/3214612554.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df.append(


,Ticker,Stock Price,Markert Capitalization,Number of Share to Buy
0,AAPL,179.21,2818740685420,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [148]:
final_df = pd.DataFrame(columns = my_columns)
for stock in stocks['Ticker'][:5]:
   symbol = stock
   api_url = f"https://cloud.iexapis.com/stable/stock/{symbol}/quote?token=pk_32ef64b2003542b6a829b8f94831c789"
   data = requests.get(api_url).json()
   final_df = final_df.append(
               pd.Series(
               [stock, data["latestPrice"], data["marketCap"], 'N/A'], 
                   index = my_columns
               ), ignore_index=True)
final_df

/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_18864/729354588.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_18864/729354588.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_18864/729354588.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_18864/729354588.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/271pb56s

,Ticker,Stock Price,Markert Capitalization,Number of Share to Buy
0,A,117.635,34746534350,N/A
1,AAL,14.885,9718416500,N/A
2,AAP,66.530,3943484650,N/A
3,AAPL,178.840,2812921065680,N/A
4,ABBV,135.190,238514321839,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [9]:
def chunk(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [10]:

symbol_group = list(chunk(stocks["Ticker"], 100))
symbol_string = []
for i in range(0, len(symbol_group)):
    symbol_string.append(','.join(symbol_group[i]))


In [15]:
final_df =  pd.DataFrame(columns = my_columns)



for symbol_str in symbol_string:  #when working with batch url use & after ?.
    batch_api_url = f"https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_str}&types=quote&token=pk_32ef64b2003542b6a829b8f94831c789"
    data = requests.get(batch_api_url).json()
    for symbol in symbol_str.split(','):
        if symbol == 'HFC' or symbol == 'VIAC' or symbol == 'WLTW' or symbol == 'DISCA': #we have to skip these there is some error maybe.
           continue
        else:
            final_df = final_df.append(
                 pd.Series(
                 [symbol, data[symbol]['quote']['latestPrice'], data[symbol]['quote']['marketCap'], 'N/A'
                 ], index = my_columns
            ),ignore_index = True)
final_df

/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/

/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/

/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/

/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/

/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/

/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/1112210979.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  final_df = final_df.append(
/var/folders/jx/

,Ticker,Stock Price,Markert Capitalization,Number of Share to Buy
0,A,117.44,34688936066,N/A
1,AAL,14.93,9747797000,N/A
2,AAP,66.07,3916218711,N/A
3,AAPL,179.21,2818740685420,N/A
4,ABBV,135.43,238937751362,N/A
...,...,...,...,...
496,YUM,132.88,37217958301,N/A
497,ZBH,130.27,27170320757,N/A
498,ZBRA,270.10,13891334294,N/A
499,ZION,30.12,4460793114,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [21]:
portfolio_size = input("Enter the value of your portfolio: ")

try:
    var =  float(portfolio_size)
    print(var)
except ValueError :
    print("That is not a number\n please try again:")
    portfolio_size = input("Enter the value of your portfolio: ")
    var =  float(portfolio_size)

Enter the value of your portfolio: 10000000
10000000.0


In [29]:
position_size = var/len(final_df.index)

for i in range(0, len(final_df.index)):
    final_df.loc[i, ["Number of Share to Buy"]] = math.floor(position_size/final_df.loc[i, ["Stock Price"]])
final_df

,Ticker,Stock Price,Markert Capitalization,Number of Share to Buy
0,A,117.44,34688936066,169
1,AAL,14.93,9747797000,1336
2,AAP,66.07,3916218711,302
3,AAPL,179.21,2818740685420,111
4,ABBV,135.43,238937751362,147
...,...,...,...,...
496,YUM,132.88,37217958301,150
497,ZBH,130.27,27170320757,153
498,ZBRA,270.10,13891334294,73
499,ZION,30.12,4460793114,662


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [63]:
writer = pd.ExcelWriter("Recommended_trades.xlsx",  engine="xlsxwriter")
final_df.to_excel(writer, "Recommended Trades", index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [64]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [65]:
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)


This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [66]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [67]:
writer.save()

/var/folders/jx/271pb56s1dndmk9yfr6rpc000000gn/T/ipykernel_22755/934276808.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
